In [42]:
import albumentations as A
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.pyplot import show
from PIL import Image
import string
import cv2
import numpy as np
import glob
import os

In [43]:
"""Välj Parent folder = mappen innan yes och no
ex D:\Projektarbete\blandat\aug\2_split_yes\train\ == D:\Projektarbete\blandat\aug\ """

data_path = Path("D:\\Projektarbete\\blandat\\aug\\")

In [44]:
#skapar en lista av alla mappar i data_path
list_of_folders = []
 
rootdir = data_path
for path in glob.glob(f'{rootdir}/*/**/', recursive=True):
    print(path)
    list_of_folders.append(path)

D:\Projektarbete\blandat\aug\2_split_no\
D:\Projektarbete\blandat\aug\2_split_no\test\
D:\Projektarbete\blandat\aug\2_split_no\train\
D:\Projektarbete\blandat\aug\2_split_no\train_aug\
D:\Projektarbete\blandat\aug\2_split_no\val\
D:\Projektarbete\blandat\aug\2_split_yes\
D:\Projektarbete\blandat\aug\2_split_yes\test\
D:\Projektarbete\blandat\aug\2_split_yes\train\
D:\Projektarbete\blandat\aug\2_split_yes\train_aug\
D:\Projektarbete\blandat\aug\2_split_yes\val\
D:\Projektarbete\blandat\aug\3_aug_no\
D:\Projektarbete\blandat\aug\3_aug_yes\


In [45]:
def checker(list, word):
    """funktion som söker efter ett visst ord i en lista, returnerar sedan svaret"""
    train_paths = []
    folderpath = [n for n, x in enumerate(list) if word in x]
    
    for i in range(len(folderpath)):
        train_paths.append(list[folderpath[i]])
    return train_paths

In [46]:
#letar upp alla mappar som heter "train" och sparar dessa sökvägar i en lista
training_folder_paths = checker(list_of_folders, 'train')

#letar upp vart ordet "no" finns i listan och returnerar sökvägen dit
no_training_path = checker(training_folder_paths,"no")
no_training_path = Path(no_training_path[0])

#letar upp vart ordet "yes" finns i listan och returnerar sökvägen dit
yes_training_path = checker(training_folder_paths,"yes")
yes_training_path = Path(yes_training_path[0])


In [47]:
#sparar ner alla filnamn som finns på destinationen
path_no = list(no_training_path.rglob('*'))
path_yes = list(yes_training_path.rglob('*'))

y0 = [0 for i in range(len(path_no))]
y1 = [1 for i in range(len(path_yes))]

CLASSES = {0:"no",1:"yes"}

CLASSES_list = []

for key in CLASSES:
    CLASSES_list.append(CLASSES[key])

In [48]:
def get_names(paths):
    """funktion som sparar ner filnamnet per bild"""
    return_list = []
    for i in range(len(paths)):
        return_list.append(Path(paths[i]).stem)
    return return_list   

In [49]:
#använder funktionen get_names för att få filnamnen på alla filer
list_of_names_no = get_names(path_no)
list_of_names_yes = get_names(path_yes)

In [50]:
print(list_of_names_no[1])
print(path_no[1])

10_no
D:\Projektarbete\blandat\aug\2_split_no\train\10_no.jpeg


In [51]:
from collections import Counter
class_paths = path_no + path_yes
y = y0 + y1
print(Counter(y))

Counter({1: 79, 0: 46})


In [52]:
def from_path_to_imglist(paths):
    """sparar ner alla sökvägar till bilder som sparas i en numpy array"""
    IMG_SIZE = (224,224)
    list_of_images = []
    for path in paths:
        img = cv2.imread(str(path))
        img = cv2.resize(img, IMG_SIZE)
        list_of_images.append(img)
    list_of_images = np.asarray(list_of_images)
    return list_of_images

In [53]:
#sökväg till bilder i numpy array
no_images = from_path_to_imglist(path_no)
yes_images = from_path_to_imglist(path_yes)

In [54]:
def aug_image_and_save_to_dict(images, list_of_names):
    """funktion som tar in alla bilderna i numpy samt alla filnamn.
    alla bilder augumenteras och dessa sparas i en lista inuti en dict per filnamn(key)"""
    dictionary = {}
    
    for k in range(len(list_of_names)):
        #roterar bilden 4 gånger 
        for i in range(4):
            augmented_image = Image.fromarray(images[k])                #skapar en bild från array
            augmented_image = augmented_image.rotate(90*(i))            #roterar bilden 90 grader
            
            try:                                                        #skapar en key utifrån filnamn och 
                dictionary[list_of_names[k]] += [augmented_image]       #sparar sedan filerna i en lista
            except:
                dictionary[list_of_names[k]] = [augmented_image]
        
        #tar de fyra roterade bilderna och flippar dem
        for i in range(len(dictionary[list_of_names[k]])):
            transform = A.Compose([A.VerticalFlip(p=1.0)])              #väljer funktionen verticalFlip
            flip = np.asarray(dictionary[list_of_names[k]][i])          #gör bilden till en np array (vi gjorde dem till image innan)
            augmented_image = transform(image=flip)['image']            #exekverar funktionen verticalFlip
            augmented_image = Image.fromarray(augmented_image)          #konverterar bild till array
            dictionary[list_of_names[k]] += [augmented_image]           #sparar i dict
        
        #tar de roterade och flippade bilderna och lägger på Clahe
        for i in range(len(dictionary[list_of_names[k]])):              
            transform = A.Compose([A.CLAHE(p=1)])                       #väljer funktionen CLAHE
            clahe = np.asarray(dictionary[list_of_names[k]][i])         #gör bilden till en np array (vi gjorde dem till image innan)
            augmented_image = transform(image=clahe)['image']           #exekverar funktionen CLAHE
            augmented_image = Image.fromarray(augmented_image)          #konverterar bild till array
            dictionary[list_of_names[k]] += [augmented_image]           #sparar i dict
    return dictionary

In [55]:
#augumenterar alla bilder och får ut dem som dict
dict_no_aug_files = aug_image_and_save_to_dict(no_images, list_of_names_no)
dict_yes_aug_files = aug_image_and_save_to_dict(yes_images, list_of_names_yes)

In [56]:
#skapar sökväg där de aug. bilderna skall sparas
no_aug_path = Path(str(path_no[0].parent.parent.parent)+"\\3_aug_no_train\\")
yes_aug_path = Path(str(path_yes[0].parent.parent.parent)+"\\3_aug_yes_train\\")

listan = [no_aug_path, yes_aug_path]

In [57]:
#kollar om mapparna finns annars så skapas de nya mapparna
for i in listan:
    if not os.path.exists(i):
        os.makedirs(i)
    else:
        print(f"{i} fungerade inte")

In [58]:
def saving_images(dictionary, paths):
    """funktion som sparar de augumenterade bilderna i rätt mappar"""
    """varje nyckel i dict är namnet på orginalfilen och sedan är value en lista på alla de augumenterade bilderna"""
    version = string.ascii_lowercase

    for key in dictionary:                                          #key = varje nyckel i dict (filnamn)
        for j in range(len(dictionary[key])):                       #j = varje bild
            filename = f"{paths}\\{key}_{str(version[j])}.jpg"
            dictionary[key][j].save(filename)

In [59]:
#spara de augumenterade bilderna
saving_yes = saving_images(dict_yes_aug_files, yes_aug_path)
saving_no = saving_images(dict_no_aug_files, no_aug_path)